In [ ]:
#from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
import time
import datetime as dt
import xarray as xr
from datetime import datetime
import pandas
import matplotlib.pyplot as plt
import numpy as np
import math
####################you will need to change some paths here!#####################
#list of input files
dir_in='f:/data/tc_wakes/database/info/'
dir_out='f:/data/tc_wakes/database/sst/'
dir_mur = 'F:/data/sst/jpl_mur/v4.1/'
dir_flux = 'F:/data/model_data/oaflux/data_v3/daily/turbulence/'
dir_cmc = 'F:/data/sst/cmc/CMC0.2deg/v2/'
#################################################################################
#from math import cos, radians
from math import sin, pi
def closest_dist(ds_in,ds_storm): 
# calculate distance closest storm point
# point given as tla,tlo.... storm is in the program
# 
#point of data
#print(tla,tlo)
# initialize distances (in degrees)
    tdim,xdim,ydim=ds_storm.lat.shape[1], ds_in.analysed_sst[0,:,0].shape[0], ds_in.analysed_sst[0,0,:].shape[0]
    dx_save=np.zeros([tdim,xdim,ydim])
    print(ds_in.analysed_sst.shape)
    dx_grid,dy_grid = np.meshgrid(ds_in.lon.values,ds_in.lat.values)
    min_dist_save = np.zeros(ds_in.analysed_sst[0,:,:].shape)*np.nan
    min_index_save = np.zeros(ds_in.analysed_sst[0,:,:].shape)*np.nan
    #for each location of the storm calculate the difference for all values in box
    for i in range(0,ds_storm.lat.shape[1]):  # all storm values
        dx = abs(dx_grid - ds_storm.lon[0,i].values)*sin(ds_storm.lat[0,i]*pi/180.)*111
        dy = abs(dy_grid - ds_storm.lat[0,i].values)*111
        dist_save=np.sqrt(dx*dx+dy*dy)
        dx_save[i,:,:]=dist_save
    #now go through each value in box and find minimum storm location/day
    for j in range(0,ds_in.lon.shape[0]):
        for i in range(0,ds_in.lat.shape[0]):
            imin = np.argmin(dx_save[:,i,j])
            min_dist_save[i,j]=dx_save[imin,i,j]
            min_index_save[i,j]=imin
    return min_dist_save,min_index_save


In [ ]:
dir_ccmp='F:/data/sat_data/ccmp/v02.0/Y'
date_1858 = dt.datetime(1858,11,17,0,0,0) # start date is 11/17/1958
dx=0.25
dy=0.25
dx_offset = -179.875
dy_offset = -78.3750


In [ ]:
#first read in 1 sst climatology just to get lat/lon
#sst climatology
filename='F:/data/sst/cmc/CMC0.2deg/v2/climatology/clim1993_2016300-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds_day=xr.open_dataset(filename,drop_variables=['analysis_error','sea_ice_fraction','sq_sst'])
ds_day = ds_day.rename({'analysed_sst':'analysed_sst_clim','mask':'mask_clim'}) #, inplace = True)            
ds_day = ds_day.where(ds_day['mask_clim'] == 1.) 
ds_day.close()


In [ ]:
def calculate_storm_mask(ds_sst,lats,lons):
    iwrap_mask = 0
    if (ds_sst.lon.max().values>170) & (ds_sst.lon.min().values<-170):
        iwrap_mask=1
    print(ds_sst.lon.min().values,ds_sst.lon.max().values)
#okay, now ds_storm is array with right lat/lon for storm so create mask now
    ds_mask = ds_sst.copy(deep=True)
    ds_mask['storm_mask']=ds_mask.analysed_sst*0
    ds_mask = ds_mask.fillna(0)
    ds_mask['storm_mask'] = ds_mask['storm_mask'].astype(int,copy=True)
    for i in range(0,lats.shape[0]):
        if lats[i]>0:   #northern hemi on right, southers on left
            lons1,lons2=lons[i]-4,lons[i]+10
        else:
            lons1,lons2=lons[i]-10,lons[i]+4
        lats1,lats2=lats[i]-10,lats[i]+10
        if i==0:
            print('lons1,lons2:',iwrap_mask,lons1.data,lons2.data)
        if lons1<-180:
            ds_mask['storm_mask'].loc[dict(lon=(ds_mask.lon < lons2) | (ds_mask.lon > lons1+360), lat=slice(lats1,lats2))] = -1
        elif lons2>180:
#                ds_sst['storm_mask'].loc[dict(lon=(ds_sst.lon < lon2) | (ds_sst.lon > lon1+360), lat=slice(lats1,lats2))] = -1
            ds_mask['storm_mask'].loc[dict(lon=(ds_mask.lon < lons2-360) | (ds_mask.lon > lons1), lat=slice(lats1,lats2))] = -1
#                ds_sst['storm_mask'].loc[dict(lon=slice(-180,lons2-360), lat=slice(lats1,lats2))] = -1
        else:
            if iwrap_mask==1:
                ds_mask.coords['lon'] = np.mod(ds_mask['lon'], 360)
                ds_mask = ds_mask.sortby(ds_mask.lon)
                ds_mask['storm_mask'].loc[dict(lon=slice(lons1+360,lons2+360), lat=slice(lats1,lats2))] = -1
                ds_mask.coords['lon'] = (ds_mask.coords['lon'] + 180) % 360 - 180
            else:
                ds_mask['storm_mask'].loc[dict(lon=slice(lons1,lons2), lat=slice(lats1,lats2))] = -1
    return ds_mask


In [ ]:
#code to create a mask for all storms
icnt = 0
for root, dirs, files in os.walk(dir_in, topdown=False):
    if root[len(dir_in):len(dir_in)+1]=='.':
        continue
    for name in files:
        if not name.endswith('.nc'):
            continue
        fname_in=os.path.join(root, name)
        print(fname_in[36:39],fname_in[31:35])
        inum_storm=int(fname_in[36:39])
        iyr_storm=int(fname_in[31:35])
        print(name,fname_in)
        dsx = xr.open_dataset(fname_in)
        lats = dsx.lat[0,:]
        lons = dsx.lon[0,:]  #lons goes from 0 to 360
        lons = (lons + 180) % 360 - 180
        dysince = dsx.time
        dsx.close()
#read in combined data        
        filename = dir_out + str(iyr_storm) + '/' + str(inum_storm).zfill(3) + '_combined_data.nc'
        ds_sst = xr.open_dataset(filename)
        ds_sst.close()

        ds_mask = calculate_storm_mask(ds_sst,lats,lons)
        
        ds_mask = ds_mask.drop(['uwnd','vwnd','date','timePlot','dbss_obml','shtfl','lhtfl','tmp2m','hum2m','analysed_sst','mask','analysed_sst_clim','mask_clim'])               

#calc distance / time
        dist,index = closest_dist(ds_sst,dsx)
        dtem=xr.DataArray(dist, coords={'lat': ds_mask.lat.values, 'lon':ds_mask.lon.values}, dims=('lat', 'lon'))
        ds_mask['dist_from_storm_km']=dtem
        dtem=xr.DataArray(index, coords={'lat': ds_mask.lat.values, 'lon':ds_mask.lon.values}, dims=('lat', 'lon'))
        ds_mask['closest_storm_index']=dtem
      

        filename = dir_out + str(iyr_storm) + '/' + str(inum_storm).zfill(3) + '_mask_data.nc'
        ds_mask.to_netcdf(filename)
        print('out:',filename)
        icnt +=1
        if inum_storm==2:
            break
    break
     # filename = dir_out + str(iyr_storm) + '/' + 'str(inum_storm)' + '_other_data.nc'
    


In [ ]:
dist,index = closest_dist(ds_sst,dsx)
print(dist.shape)
print(ds_mask)
dtem=xr.DataArray(dist, coords={'lat': ds_mask.lat.values, 'lon':ds_mask.lon.values}, dims=('lat', 'lon'))
ds_mask['dist_from_storm_km']=dtem
dtem=xr.DataArray(index, coords={'lat': ds_mask.lat.values, 'lon':ds_mask.lon.values}, dims=('lat', 'lon'))
ds_mask['closest_storm_index']=dtem
ds_mask

In [ ]:
ds_mask
#ds_mask.storm_mask[-1,:,:].plot()
#ds_sst.analysed_sst[0,:,:].plot()

In [ ]:
#check output data
iwrap=0
iyr_storm,inum_storm=2002,5
filename = dir_out + str(iyr_storm) + '/' + str(inum_storm).zfill(3) + '_combined_data.nc'
ds = xr.open_dataset(filename)
ds.close()
ds_sst = ds.copy(deep=True)   #make a deep copy
dsx = xr.open_dataset('F:/data/tc_wakes/database/info/'+str(iyr_storm)+'/'+ str(inum_storm).zfill(3) +'annual_storm_info.nc')
lats = dsx.lat[0,:]
lons = dsx.lon[0,:]  #lons goes from 0 to 360
lons = (lons + 180) % 360 - 180
dsx.close()
filename = dir_out + str(iyr_storm) + '/' + str(inum_storm).zfill(3) + '_mask_data.nc'
ds_mask = xr.open_dataset(filename)
ds_mask.close()
print(ds_mask)



In [ ]:
dsx.lat

In [ ]:

#md,mi,dx= closest_dist(ds,dsx)

In [ ]:
print(dx_save.shape)
plt.pcolormesh(min_index_save)
#plt.pcolormesh(dx_grid[:,:])
#print(md.shape,md[10,10],md[60,60],md[70,70],md[80,80])
#plt.pcolormesh(md,vmin=0,vmax=5000)
#print(dx_grid[0,:])

In [ ]:
plt.pcolormesh(ds.analysed_sst[10,:,:])
#plt.pcolormesh(ds.lon,ds.lat,ds.analysed_sst[10,:,:])
#plt.plot(lons,lats,'red')

In [ ]:
plt.pcolormesh(ds_mask.storm_mask[10,:,:])
#plt.pcolormesh(ds_mask.lon,ds_mask.lat,ds_mask.storm_mask[10,:,:])
#plt.plot(lons,lats,'red')

In [ ]:
ds.analysed_sst[0,:,:].plot()

In [ ]:
ds_subset.analysed_sst[0,:,:].plot()

In [ ]:
ds_sst['storm_mask']=ds_sst.analysed_sst*0
ds_sst['storm_mask'] = ds_sst['storm_mask'].astype(int,copy=True)
for i in range(0,lats.shape[0]):
    if lats[i]>0:   #northern hemi on right, southers on left
        ds_sst['storm_mask'].loc[dict(lon=slice(lons[i]-4,lons[i]+10), lat=slice(lats[i]-2,lats[i]+2))] = -1
    else:
        ds_sst['storm_mask'].loc[dict(lon=slice(lons[i]-10,lons[i]+4), lat=slice(lats[i]-2,lats[i]+2))] = -1        
ds_sst.drop(['uwnd','vwnd','date','timePlot','dbss_obml','shtfl','lhtfl','tmp2m','hum2m','analysed_sst','mask','analysed_sst_clim','mask_clim'])

        

In [ ]:
ds_sst.drop(['uwnd','vwnd','date','timePlot','dbss_obml','shtfl','lhtfl','tmp2m','hum2m','analysed_sst','mask','analysed_sst_clim','mask_clim'])

In [ ]:
ds.analysed_sst[0,:,:].plot()